# Plots

In [17]:
import pickle
from pathlib import Path

import glob
import sys
import torch

from collections import defaultdict
import pandas as pd

In [18]:
def optimality_gap(bk_sol, opt_sol):
    return ((bk_sol - opt_sol) / bk_sol).mean().item() * 100

In [19]:

# Get all files inside ../results iteratively with .pkl extension
files = glob.glob("../results/**/*.pkl", recursive=True)

# get only tsp50 files
files = [file for file in files if "cvrp50" in file]

print(files)



['../results/eval_methods_pareto/cvrp50/symnco-cvrp50/results.pkl', '../results/eval_methods_pareto/cvrp50/am-cvrp50-sm/results.pkl', '../results/eval_methods_pareto/cvrp50/am-critic-cvrp50/results.pkl', '../results/eval_methods_pareto/cvrp50/am-cvrp50/results.pkl', '../results/eval_methods_pareto/cvrp50/pomo-cvrp50/results.pkl', '../results/eval_methods_pareto/cvrp50/am-cvrp50-sm-xl/results.pkl']


## Solver files

These contain the BK solutions (best know), from which we compute the gaps

We need to load depending on the env

In [21]:
# Concorde
def load_solver_file_tsp(num_nodes):
    solver_path = Path("../solver/tsp")
    solver_files = glob.glob(str(solver_path / "*tsp*.pkl"), recursive=True)
    # load pickle of first solver file
    file_ = solver_path / f"tsp_{num_nodes}.pkl"
    with open(file_, 'rb') as f:
        costs = pickle.load(f)
    print("Concorde time for {} nodes: {:.2f}s".format(num_nodes, costs[0]))
    print("Average cost: {:.4f}".format(sum(costs[1])/len(costs[1])))

    return torch.Tensor(costs[1])



# HGS
def load_solver_file_vrp(num_nodes):

    solver_path = Path("../solver/vrp")

    dir_ = solver_path / f"{num_nodes}_test_seed_1234"

    solver_files = sorted(glob.glob(str(dir_ / "*.txt")))
    # take the interger of file number and order by it
    solver_files = sorted(solver_files, key=lambda x: int(x.split("/")[-1].split(".")[0]))
    costs = []
    for solver_file in solver_files:
        with open(solver_file, "r") as f:
            # get lines as dict
            lines = f.readlines()
            lines = [line.strip().split() for line in lines]
            lines = {line[0]: line[1:] for line in lines}
            costs.append(float(lines['obj_val:'][0]))
    print("Average cost: {:.4f}".format(sum(costs)/len(costs)))
    return torch.Tensor(costs)


# Example
costs = load_solver_file_vrp(50)


Average cost: 10.3661


In [22]:
# Make plot

experiments = ['greedy', 'augment', 'sampling', 
               'greedy_multistart', 'greedy_multistart_augment']

## Save results in table

In [23]:

# Get all files inside ../results iteratively with .pkl extension
files = glob.glob("../results/**/*.pkl", recursive=True)

# # TSP50
# costs = load_solver_file_tsp(50)
# files = [file for file in files if "tsp50" in file]
# results_name = "tsp50"

# TSP20
costs = load_solver_file_tsp(20)
files = [file for file in files if "tsp20" in file]
results_name = "tsp20"


# # CVRP50
# costs = load_solver_file_vrp(50)
# files = [file for file in files if "cvrp50" in file]
# results_name = "cvrp50"



# # CVRP20
# costs = load_solver_file_vrp(50)
# files = [file for file in files if "cvrp50" in file]
# results_name = "cvrp50"


# Make a CSV table of all results
csv_results = defaultdict(list)


for f in files:
    data = pickle.load(open(f, "rb"))
    filename = "plots/" + f.split("/")[-3] + "/" + f.split("/")[-2]

    model_name = filename.split("/")[-1]

    for experiment in experiments:
        
        for exp in data[experiment]:

            costs_model = -exp['rewards']
            gap = optimality_gap(costs_model, costs)

            costs_model_mean = costs_model.mean().item()

            time = exp['inference_time']

            csv_results['model'].append(model_name)

            # exp name is exp['exp_kwargs'] combo of key and values if any
            kwargs = ""
            for key, value in exp['exp_kwargs'].items():
                kwargs += f"{key}_{value}_"
            csv_results['experiment'].append(experiment)
            csv_results['kwargs'].append(kwargs)
            csv_results['gap'].append(gap)
            csv_results['costs'].append(costs_model_mean)
            csv_results['time'].append(time)
    


# Save as markdown table

df = pd.DataFrame(csv_results)
# df.to_csv("results.csv", index=False)


# # SAve as markdown table
# df.to_markdown("results.md", index=False)


# to html
# df.to_html("results.html", index=False)

# to excel
df.to_excel(f"tables/{results_name}2.xlsx", index=False)

Concorde time for 20 nodes: 1132.89s
Average cost: 3.8357
